In [5]:
import pandas as pd


input_folder = 'csv_inputs'
csvs = [
         '2022h1'
        ,'2022h2'
        ]

In [6]:
# Combine csvs

dfs = []
for i in csvs:
    tmp = pd.read_csv(input_folder + '/'+ i + '.csv')
    dfs.append(tmp)

df = pd.concat(dfs)

# Sort the data by 'number' in ascending order
df = df.sort_values(by='number')

display(df.head(10))

### Check Base Fee Deltas by Interval

In [ ]:
# # Define the block intervals
# intervals = [1, 5, 25, 50, 75, 100, 125, 150]

# # Initialize a dictionary to hold the differences
# diffs = {}

# # Calculate the base price difference for each interval
# for interval in intervals:
#     df[f'diff_{interval}'] = df['base_fee_gwei'].diff(periods=interval)
#     diffs[interval] = df[f'diff_{interval}'].dropna()

# diffs

# import matplotlib.pyplot as plt

# # Create a figure and axis
# fig, ax = plt.subplots(figsize=(12, 8))

# # Convert the dictionary to a DataFrame for easier plotting
# df_diffs = pd.DataFrame(diffs)

# # Create boxplots
# df_diffs.boxplot(ax=ax, showfliers=False)

# # Set the title and labels
# ax.set_title('Distribution of Base Fee Price Differences at Different Block Intervals')
# ax.set_xlabel('Block Intervals')
# ax.set_ylabel('Base Fee Price Difference (Gwei)')

# # Show the plot
# plt.show()

### Identify Spikes

In [ ]:
# Calculate the percentage change in base fee from one block to the next
df['pct_change'] = df['base_fee_gwei'].pct_change() * 100

# Initialize an empty list to hold the start and end indices of each spike
spikes = []

# Initialize a variable to hold the start index of the current spike
start_index = None

# Iterate over the DataFrame
for i in range(len(df)):
    # If a spike has not started yet
    if start_index is None:
        # If the base fee increases by 10% or more, start a new spike
        if df['pct_change'].iloc[i] >= 10:
            start_index = i
    # If a spike has already started
    else:
        # If the base fee returns to within 10% of the base fee at the start of the spike, end the current spike
        if abs(df['base_fee_gwei'].iloc[i] - df['base_fee_gwei'].iloc[start_index]) <= df['base_fee_gwei'].iloc[start_index] * 0.1:
            spikes.append((start_index, i))
            start_index = None

# Convert the list of spikes to a DataFrame
df_spikes = pd.DataFrame(spikes, columns=['Start Index', 'End Index'])

# Calculate the duration of each spike (in blocks)
df_spikes['Duration (blocks)'] = df_spikes['End Index'] - df_spikes['Start Index']

# Calculate the duration of each spike (in minutes) assuming a block time of 12 seconds
df_spikes['Duration (minutes)'] = df_spikes['Duration (blocks)'] * 12 / 60

# Add start and end block numbers and times to the DataFrame
df_spikes['Start Block Number'] = df.iloc[df_spikes['Start Index']]['number'].values
df_spikes['End Block Number'] = df.iloc[df_spikes['End Index']]['number'].values
df_spikes['Start Time'] = df.iloc[df_spikes['Start Index']]['time'].values
df_spikes['End Time'] = df.iloc[df_spikes['End Index']]['time'].values

# Calculate the starting base fee and peak base fee of each spike
df_spikes['Starting Base Fee'] = df.iloc[df_spikes['Start Index']]['base_fee_gwei'].values
df_spikes['Peak Base Fee'] = df.iloc[df_spikes['Start Index']:df_spikes['End Index']].apply(lambda x: df['base_fee_gwei'].iloc[x['Start Index']:x['End Index']].max(), axis=1)

# Calculate the percentage gain of each spike
df_spikes['Percentage Gain'] = (df_spikes['Peak Base Fee'] - df_spikes['Starting Base Fee']) / df_spikes['Starting Base Fee'] * 100

# Display the DataFrame
df_spikes


,Start Index,End Index,Duration (blocks),Duration (minutes)
0,4,6,2,0.4
1,7,8,1,0.2
2,15,16,1,0.2
3,28,30,2,0.4
4,36,37,1,0.2
...,...,...,...,...
187931,2391997,2391998,1,0.2
187932,2392004,2392005,1,0.2
187933,2392008,2392009,1,0.2
187934,2392011,2392012,1,0.2
